In [18]:
from pathlib import Path

from tensorboard.backend.event_processing import event_accumulator
import pandas as pd


def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    assert all(
        s in ea.Tags()["scalars"] for s in scalars
    ), "some scalars were not found in the event accumulator"
    return {k: pd.DataFrame(ea.Scalars(k)) for k in scalars}

# all dirs like data/fomo/output/CoPrompt_
dirs = list(Path('data/fomo/output').glob('CoPrompt_*'))
rs = []
for dir in dirs:
    seed1 = dir / 'train_base/ucf101/shots_16/CoPrompt/coprompt/seed1/'
    tensorboard = seed1 / 'tensorboard'
    clog = seed1 / 'clog.txt'
    
    if not clog.exists():
        continue
        
    with open(clog, 'r') as f:
        lines = f.readlines()
        if len(lines) < 2:
            continue
        p = lines[-1][4:-5]
    
    # read from tensorboard)
    try:
        df = parse_tensorboard(tensorboard, ['test/accuracy'])
        test_acc = df['test/accuracy'].iloc[-1]['value']
    except Exception as e:
        print(e)
        continue
        
    rs.append((test_acc, p, str(dir)))
    
    
for test_acc, p, m in sorted(rs, reverse=True):
    print(f"[{test_acc}], {p}")
    print(m)



some scalars were not found in the event accumulator
some scalars were not found in the event accumulator
some scalars were not found in the event accumulator
some scalars were not found in the event accumulator
some scalars were not found in the event accumulator
some scalars were not found in the event accumulator
[77.24922180175781], katelyn davis doing "#  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt4_maxepoch16
[76.8355712890625], freaking iit alex egan #  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt5_maxepoch8
[76.78386688232422], awesome instructional  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt2_maxepoch16
[76.42192077636719], tutorial instructional  
data/fomo/output/CoPrompt_m0.0_wstd0.004_nctxt2_maxepoch16
[75.69803619384766], jurphil practising atmultitasking stingly stingly individual  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt8_maxepoch16
[75.64633178710938], iaccredited teaches  
data/fomo/output/CoPrompt_m0.0_wstd0.004_nctxt3_maxepoch16
[75.59461975097656

In [6]:
def convert_tb_data(root_dir, sort_by=None):
    """Convert local TensorBoard data into Pandas DataFrame.
    
    Function takes the root directory path and recursively parses
    all events data.    
    If the `sort_by` value is provided then it will use that column
    to sort values; typically `wall_time` or `step`.
    
    *Note* that the whole data is converted into a DataFrame.
    Depending on the data size this might take a while. If it takes
    too long then narrow it to some sub-directories.
    
    Paramters:
        root_dir: (str) path to root dir with tensorboard data.
        sort_by: (optional str) column name to sort by.
    
    Returns:
        pandas.DataFrame with [wall_time, name, step, value] columns.
    
    """
    import os
    import pandas as pd
    from tensorflow.python.summary.summary_iterator import summary_iterator

    def convert_tfevent(filepath):
        return pd.DataFrame([
            parse_tfevent(e) for e in summary_iterator(filepath) if len(e.summary.value)
        ])

    def parse_tfevent(tfevent):
        return dict(
            wall_time=tfevent.wall_time,
            name=tfevent.summary.value[0].tag,
            step=tfevent.step,
            value=float(tfevent.summary.value[0].simple_value),
        )
    
    columns_order = ['wall_time', 'name', 'step', 'value']
    
    out = []
    for (root, _, filenames) in os.walk(root_dir):
        for filename in filenames:
            if "events.out.tfevents" not in filename:
                continue
            file_full_path = os.path.join(root, filename)
            out.append(convert_tfevent(file_full_path))

    # Concatenate (and sort) all partial individual dataframes
    all_df = pd.concat(out)[columns_order]
    if sort_by is not None:
        all_df = all_df.sort_values(sort_by)
        
    return all_df.reset_index(drop=True)

if __name__ == "__main__":
    dir_path = "/home/kretyn/projects/ai-traineree/runs/"
    exp_name = "CartPole-v1_2021-01-26_11:02"
    df = convert_tb_data(f"{dir_path}/{exp_name}")
    
    print(df.head())

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import tensorboard as tb
tb.__version__

'2.17.0a20240412'